In [1]:
import tensorflow as tf
from tensorflow import keras
from skimage.io import imread, imshow
from skimage.transform import resize
from tqdm import tqdm 
import cv2
import os
import numpy as np

## Numpy and Scimage Method for Data 

In [2]:
img_height=512
img_width=512
channel=1
input_shape=(img_height,img_width,channel)

In [4]:
TRAIN_PATH = '/Users/matthewavaylon/Data_G/Originals/'
label_path= '/Users/matthewavaylon/Data_G/Labeled/'

In [5]:
train_ids = next(os.walk(TRAIN_PATH))[1]
train_ids=sorted(train_ids)

In [6]:
label_ids = next(os.walk(label_path))[1]
label_ids=sorted(label_ids)

In [7]:
X_train = np.zeros((len(train_ids), img_height, img_width,channel), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), img_height, img_width,channel), dtype=np.uint8)

In [8]:
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):   
    path = TRAIN_PATH + id_
    img = imread(path + "/" + id_ + '.png') 
    img = np.expand_dims(img, axis =-1)
    img = resize(img, (img_height, img_width), mode='constant', preserve_range=True)
    X_train[n] = img  #Fill empty X_train with values from img

100%|██████████| 512/512 [00:06<00:00, 83.51it/s]


In [9]:
for n, id_ in tqdm(enumerate(label_ids), total=len(label_ids)):   
    path = label_path + id_
    img = imread(path + "/" + id_ + '.png')
    img = np.expand_dims(img, axis =-1)
    #img2 = cv2.merge((img,img,img))
    img = resize(img, (img_height, img_width), mode='constant', preserve_range=True)
    Y_train[n] = img  #Fill empty X_train with values from img

100%|██████████| 512/512 [00:05<00:00, 99.17it/s] 


## Model

In [11]:
input_=tf.keras.layers.Input(shape=input_shape)
s = tf.keras.layers.Lambda(lambda x: x / 255)(input_)

In [12]:
c1 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2), strides=2)(c1)

c2 = tf.keras.layers.Conv2D(filters=32,kernel_size= (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2),strides=2)(c2)
 
c3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2),strides=2)(c3)

c4 = tf.keras.layers.Conv2D(filters=128,  kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Conv2D(filters=128,  kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2, 2),strides=2)(c4)


c5 = tf.keras.layers.Conv2D(filters=256,  kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Conv2D(filters=256,  kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

In [13]:
u1 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(2, 2), strides=(2, 2), padding='same')(c5)
u1 = tf.keras.layers.concatenate([u1, c4])
c6 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u1)
c6 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u2 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(2, 2), strides=(2, 2), padding='same')(c6)
u2 = tf.keras.layers.concatenate([u2, c3])
c7 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u2)
c7 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u3 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(2, 2), strides=(2, 2), padding='same')(c7)
u3 = tf.keras.layers.concatenate([u3, c2])
c8 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u3)
c8 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u4 = tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=(2, 2), strides=(2, 2), padding='same')(c8)
u4 = tf.keras.layers.concatenate([u4, c1])
c9 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u4)
c9 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

In [14]:
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

In [19]:
model = tf.keras.Model(inputs=input_, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512, 512, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 16) 160         lambda[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 16) 2320        conv2d[0][0]                     
______________________________________________________________________________________________

In [20]:
model.fit(X_train, Y_train, batch_size=32, epochs=5)

Train on 512 samples
Epoch 1/5
192/512 [==========>...................] - ETA: 5:41 - loss: -1142.0117 - accuracy: 0.0852

KeyboardInterrupt: 